# Análise exploratória

## Carregamento e preparação dos dados

### Obtenção e carregamento de dados  

In [ ]:
#importação de bibliotecas
import pandas as pd

import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt



In [ ]:
# Importação dos dados
df_dataset = pd.read_csv('../Data/Consolidated/data_encoded.csv', sep=';', encoding='utf-8', on_bad_lines='skip')

# Carregar dicionário de dados
df_dictionary = pd.read_csv("../Data/Consolidated/dictionary_consolidated.csv", encoding="ISO-8859-1", sep=";", on_bad_lines='skip')

### Informações sobre o dataset

In [ ]:
#Exibir informações sobre o dataframe
df_dataset.info()

#Exibir as primeiras linhas do dataframe
df_dataset.describe()

### Visualização inicial dos dados

In [ ]:
#Exibir as primeiras linhas do dataframe
df_dataset.head()

### Definição de índices

In [ ]:
# Colocar a coluna 'Survey_id' na primeira posição
df_dataset = df_dataset[['Survey_id'] + [col for col in df_dataset.columns if col != 'Survey_id']]

# Definir o índice do dataframe como a coluna 'Survey_id'
df_dataset.set_index('Survey_id', inplace=True)

# Exibir as primeiras linhas do dataframe
df_dataset.head()

## Análise de variáveis

In [ ]:


# 3) Inventário de variáveis
meta = df_dictionary[['Variable', 'Type', 'Question']].rename(columns={
    'Variable':'column', 'Type':'type', 'Question':'description'
})
print("Metadados das variáveis:")
display(meta)

# 4) Qualidade dos dados

mapping = dict(zip(meta['column'], meta['description']))

# 4.1) Informações gerais
print("\nInformações gerais do DataFrame:")
df_dataset.info()

# 4.2) Valores faltantes
missing = df_dataset.isna().sum().to_frame('n_missing')
missing['pct_missing'] = (missing['n_missing'] / len(df_dataset)) * 100

missing = (
    missing
    .reset_index()
    .rename(columns={'index':'column'})
    # 4) Adiciona a pergunta correspondente via mapeamento
    .assign(question=lambda df: df['column'].map(mapping))
    # 5) Reordena as colunas e ordena pelo % de faltantes
    .loc[:, ['column','question','n_missing','pct_missing']]
    .sort_values('pct_missing', ascending=False)
)
display(missing.head(50))

# 4.3) Duplicatas
n_dup = df_dataset.duplicated().sum()
print(f"\nRegistros duplicados: {n_dup}")

# 5) Estatísticas descritivas
# 5.1) Numéricas
print("\nEstatísticas descritivas (numéricas):")
display(df_dataset.select_dtypes(include=[np.number]).describe().T)

# 5.2) Categóricas
cat_cols = df_dataset.select_dtypes(include=['object', 'category']).columns
print("\nContagem das 5 principais categorias em cada variável categórica:")
for col in cat_cols:
    vc = df_dataset[col].value_counts(dropna=False).head(5)
    print(f"\n{col}:")
    print(vc.to_string())
